In [1]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents.initialize import initialize_agent
from langchain.agents import Tool
from langchain_google_genai import GoogleGenerativeAI


/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wrapper = DuckDuckGoSearchAPIWrapper(time="d")
# create a search agent
search = DuckDuckGoSearchResults(api_wrapper=wrapper)
tools = [search]

In [3]:
# from dotenv import load_dotenv
# load_dotenv()
# wrapper = DuckDuckGoSearchAPIWrapper(time="d")
# # create a search agent
# search = DuckDuckGoSearchResults(api_wrapper=wrapper)
# tools = [
#     Tool(
#         name="Search",
#         func=search.run,
#         description="useful for when you need to answer questions about current events",
#     )
# ]

In [4]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [5]:
llm = GoogleGenerativeAI(model="gemini-pro")

In [6]:
llm_chain = LLMChain(llm=GoogleGenerativeAI(model="gemini-pro"), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [7]:
agent_chain.run(input="How many people live in canada?")

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I do not know the answer to this question
Action: duckduckgo_results_json
Action Input: how many people live in canada

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:



Observation: [snippet: Brantford (2021 population: 104,688) is a city in Ontario, Canada, founded on the Grand River in Southwestern Ontario.It is surrounded by Brant County, but is politically separate with a municipal government of its own that is fully independent of the county's municipal government.. Brantford is situated on the Haldimand Tract, and is named after Joseph Brant, a Mohawk leader, soldier, farmer ..., title: Brantford - Wikipedia, link: https://en.wikipedia.org/wiki/Brantford], [snippet: Halifax is the capital and largest municipality of the Canadian province of Nova Scotia, and the largest municipality in Atlantic Canada.As of 2022, it is estimated that the population of the Halifax CMA was 480,582, with 348,634 people in its urban area. The regional municipality consists of four former municipalities that were amalgamated in 1996: Halifax, Dartmouth, Bedford, and Halifax County., title: Halifax, Nova Scotia - Wikipedia, link: https://en.wikipedia.org/wiki/Halifax,

'I am unable to answer this question'

In [8]:
agent_chain.run(input="capital of this coutry?")



> Entering new AgentExecutor chain...
Thought: The question is "capital of this coutry?". Since I don't know what country the user is referring to, I should ask for clarification.
Action: respond with text
Action Input: "What country are you referring to?"
Observation: respond with text is not a valid tool, try one of [duckduckgo_results_json].
Thought:

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I am unable to answer this question`